In [121]:
import cvxpy as cp
import numpy as np
from numpy.linalg import pinv, inv, norm, det
from scipy.linalg import eig
import networkx as nx
import math

In [127]:
n = 3
p = 100

W = np.random.rand(n, n)
W = np.round((W + W.T) / 2)
D = np.diag(np.sum(W, axis=0))
L_orig = D - W 
L_orig = np.true_divide(L_orig , np.trace(L_orig) * n)
[Lam, U] = eig(L_orig)
Lam, U = Lam.real, U.real

mu = np.zeros((n))
sigma = pinv(np.diag(Lam))
h = np.random.multivariate_normal(mu, sigma, p).T

weights = np.random.randint(1,10,size=(n,n))
weights = np.round((weights + weights.T) /2)


sigma_eps = 1e-5
X = U @ h + np.random.multivariate_normal(np.zeros((n)), sigma_eps*np.eye(n), p).T

In [128]:
def gl_logdet(X, W, weights, lamda, sigma, p, n):
    
    T = np.diag(np.sum(weights, axis=0))
    
    Lpre = cp.Variable((n,n))
    
    obj = cp.Minimize((1/p)*cp.trace(Lpre @ X @ X.T) + cp.atoms.log_det(Lpre) + (lamda/p)*norm(W, 1))
    
    constraints = [Lpre == T - W + (1/(np.sum(sigma**2)))*np.eye(n)]
    
    constraints += [Lpre >> 0]
    constraints += [np.sum(sigma**2) > 0]
    for i in range(0,n):
        for j in range(0,n):
            if i==j:
                constraints += [weights[i,j] == 0]
            else:
                constraints += [weights[i,j] >= 0]
                
    prob = cp.Problem(obj, constraints)
    
    p_star = prob.solve()
    L_pre = Lpre.value
    
    return L_pre

In [129]:
lamda = 1e-2

L_pre = gl_logdet(X, W, weights, lamda, sigma, 0.1, 3)

AttributeError: 'numpy.bool_' object has no attribute 'parameters'